In [1]:
import json
import numpy as np
import re

In [2]:
root_folder = '/home/lyt/code/concept_vqa'
data_folder = '{}/dataTVQA'.format(root_folder)
tvqa_folder = '/home/data3/lyt/TextVQA'

In [3]:
#=======================================

In [4]:
norm_text = True

In [5]:
split_name = 'val'
jsdata = json.load(open('{}/TextVQA_0.5_{}.json'.format(tvqa_folder, split_name),'r'))
print(jsdata.keys())

dict_keys(['dataset_type', 'dataset_name', 'dataset_version', 'data'])


In [6]:
data = jsdata['data']
print(len(data),type(data))

5000 <class 'list'>


In [7]:
#------save qid:ocr dict------

In [8]:
# borrow from vqaEval.py
m_contractions = {
    "aint": "ain't", "arent": "aren't", "cant": "can't", "couldve": "could've",
    "couldnt": "couldn't", "couldn'tve": "couldn't've",
    "couldnt've": "couldn't've", "didnt": "didn't", "doesnt": "doesn't",
    "dont": "don't", "hadnt": "hadn't", "hadnt've": "hadn't've",
    "hadn'tve": "hadn't've", "hasnt": "hasn't", "havent": "haven't",
    "hed": "he'd", "hed've": "he'd've", "he'dve": "he'd've", "hes": "he's",
    "howd": "how'd", "howll": "how'll", "hows": "how's", "Id've": "I'd've",
    "I'dve": "I'd've", "Im": "I'm", "Ive": "I've", "isnt": "isn't",
    "itd": "it'd", "itd've": "it'd've", "it'dve": "it'd've", "itll": "it'll",
    "let's": "let's", "maam": "ma'am", "mightnt": "mightn't",
    "mightnt've": "mightn't've", "mightn'tve": "mightn't've",
    "mightve": "might've", "mustnt": "mustn't", "mustve": "must've",
    "neednt": "needn't", "notve": "not've", "oclock": "o'clock",
    "oughtnt": "oughtn't", "ow's'at": "'ow's'at", "'ows'at": "'ow's'at",
    "'ow'sat": "'ow's'at", "shant": "shan't", "shed've": "she'd've",
    "she'dve": "she'd've", "she's": "she's", "shouldve": "should've",
    "shouldnt": "shouldn't", "shouldnt've": "shouldn't've",
    "shouldn'tve": "shouldn't've", "somebody'd": "somebodyd",
    "somebodyd've": "somebody'd've", "somebody'dve": "somebody'd've",
    "somebodyll": "somebody'll", "somebodys": "somebody's",
    "someoned": "someone'd", "someoned've": "someone'd've",
    "someone'dve": "someone'd've", "someonell": "someone'll",
    "someones": "someone's", "somethingd": "something'd",
    "somethingd've": "something'd've", "something'dve": "something'd've",
    "somethingll": "something'll", "thats": "that's", "thered": "there'd",
    "thered've": "there'd've", "there'dve": "there'd've",
    "therere": "there're", "theres": "there's", "theyd": "they'd",
    "theyd've": "they'd've", "they'dve": "they'd've", "theyll": "they'll",
    "theyre": "they're", "theyve": "they've", "twas": "'twas",
    "wasnt": "wasn't", "wed've": "we'd've", "we'dve": "we'd've",
    "weve": "we've", "werent": "weren't", "whatll": "what'll",
    "whatre": "what're", "whats": "what's", "whatve": "what've",
    "whens": "when's", "whered": "where'd", "wheres": "where's",
    "whereve": "where've", "whod": "who'd", "whod've": "who'd've",
    "who'dve": "who'd've", "wholl": "who'll", "whos": "who's",
    "whove": "who've", "whyll": "why'll", "whyre": "why're", "whys": "why's",
    "wont": "won't", "wouldve": "would've", "wouldnt": "wouldn't",
    "wouldnt've": "wouldn't've", "wouldn'tve": "wouldn't've",
    "yall": "y'all", "yall'll": "y'all'll", "y'allll": "y'all'll",
    "yall'd've": "y'all'd've", "y'alld've": "y'all'd've",
    "y'all'dve": "y'all'd've", "youd": "you'd", "youd've": "you'd've",
    "you'dve": "you'd've", "youll": "you'll", "youre": "you're",
    "youve": "you've"}
m_manual_map = {'none': '0', 'zero': '0', 'one': '1', 'two': '2', 'three': '3',
                'four': '4', 'five': '5', 'six': '6', 'seven': '7',
                'eight': '8', 'nine': '9', 'ten': '10'}
m_articles = ['a', 'an', 'the']
m_period_strip = re.compile("(?!<=\d)(\.)(?!\d)")
m_comma_strip = re.compile("(\d)(\,)(\d)")
m_punct = [';', r"/", '[', ']', '"', '{', '}', '(', ')', '=', '+',
           '\\', '_', '-', '>', '<', '@', '`', ',', '?', '!']

def process_punct(in_text):
    out_text = in_text
    for p in m_punct:
        if (p + ' ' in in_text or ' ' + p in in_text
                or re.search(m_comma_strip, in_text) != None):
            out_text = out_text.replace(p, '')
        else:
            out_text = out_text.replace(p, ' ')
    out_text = m_period_strip.sub("", out_text, re.UNICODE)
    return out_text

def process_digit_article(in_text):
    out_text = []
    for word in in_text.lower().split():
        if word not in m_articles:
            word = m_manual_map.setdefault(word, word)
            word = m_contractions.setdefault(word, word)
            out_text.append(word)
    return ' '.join(out_text)

def normText(text):
    text = process_punct(text)
    text = process_digit_article(text)
    return text

def stripLower(text):
    text = text.replace('\n', ' ').replace('\t', ' ').strip().lower()
    return text

In [9]:
qid_ocr = {}
for pair in data:
    qid = pair['question_id']
    if qid not in qid_ocr:
        all_ocr = [stripLower(ocr) for ocr in pair['ocr_tokens']]
        if norm_text:
            all_ocr = [normText(ocr) for ocr in all_ocr]
        qid_ocr[qid] = [ocr for ocr in all_ocr if ocr != '']
json.dump(qid_ocr, open('{}/qid_ocr_{}.json'.format(data_folder, split_name),'w'))
len(qid_ocr)

5000

In [10]:
# qid_ocr[39602]